In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import itertools as it
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, cross_val_predict, RandomizedSearchCV
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier, AdaBoostRegressor, AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_regression
import warnings
np.warnings.filterwarnings('ignore')

In [6]:
# Loading the data
train = pd.read_csv('kaggle_train.csv')
test = pd.read_csv('kaggle_test.csv')

# Separate into test and train
y_train = train.y
X_train = train.drop("y", axis = 1)

X_test = test

X_train_columns = X_train.columns
X_test_columns = X_test.columns

# Scale the data
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train_columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train_columns)

# Impute the missing values
X_imputed = pd.DataFrame(KNNImputer(n_neighbors = 8).fit_transform(X_train), columns = X_train.columns)
X_test_imputed = pd.DataFrame(KNNImputer(n_neighbors = 8).fit_transform(X_test), columns = X_test.columns)

# SelectKBest to select the best 350 features of the dataset
# Fit using the imputed X_train data
# Get_support() gives an array with Booleans with True being the predictors selected and False being the predictors removed
selected_features = SelectKBest(f_regression, k = 350).fit(X_imputed, y_train).get_support()

In [7]:
# Making a new dataframe with all the predictors that are selected
X_train_small = X_imputed[X_imputed.columns[selected_features]]

In [8]:
X_test_small = X_test_imputed[X_test_imputed.columns[selected_features]]

In [9]:
# Coarse search
param_grid = {
    'max_depth': list(range(0, 200, 20)),
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
    'subsample': [0.25, 0.5, 0.75, 1.0],
    'reg_lambda': [0, 1, 5, 10],
    'n_estimators': range(0, 2000, 400),
    'gamma': [0, 10]
}

cv = KFold(n_splits=2, shuffle=True, random_state=1)

lgb_coarse_grid_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(random_state=1),
    param_distributions=param_grid,
    verbose=1,
    n_jobs=-1,
    cv=cv,
    scoring='neg_mean_absolute_error',
    n_iter=50,
    random_state=1
)

lgb_coarse_grid_result = lgb_coarse_grid_search.fit(X_train_small, y_train)

# Store results in a DataFrame
lgb_cv_results = pd.DataFrame(lgb_coarse_grid_result.cv_results_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[LightGBM] [Warning] Unknown parameter: gamma


In [10]:
print('Best MAE Score Through Grid Search : %.3f'%lgb_coarse_grid_search.best_score_)
print('Best Parameters : ',lgb_coarse_grid_search.best_params_)

Best MAE Score Through Grid Search : -5.161
Best Parameters :  {'subsample': 0.75, 'reg_lambda': 10, 'n_estimators': 400, 'max_depth': 80, 'learning_rate': 0.01, 'gamma': 10}


In [9]:
lgb_model = lgb.LGBMRegressor(random_state=1, verbose = 1, n_estimators = 400, subsample = 0.75, reg_lambda = 10,
                              max_depth = 80, learning_rate = 0.01, gamma = 10,
                              n_jobs=-1).fit(X_train_small,y_train)

pred = lgb_model.predict(X_test_small)

NameError: name 'lgb' is not defined

In [48]:
submission_LGBM = pd.read_csv('submission_LGBM.csv')
submission_LGBM.shape

(5380, 2)

In [49]:
submission_LGBM.dropna(inplace = True)
submission_LGBM.shape

(4403, 2)

In [50]:
submission_LGBM.id = submission_LGBM.id.apply(np.int32)
submission_LGBM.id.dtype

dtype('int32')

In [51]:
submission_LGBM.to_csv('submission_LGBM1.csv', index=False)